### 【 윤동주 시인의 시 제목과 내용 추출 및 저장 】
- csv 파일로 저장

In [2]:
## 모듈로딩 
from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup

In [3]:
## URL
WIKI_URL = "https://ko.wikisource.org/wiki/%EC%A0%80%EC%9E%90:%EC%9C%A4%EB%8F%99%EC%A3%BC"

## Request 객체에 User-Agent 헤더 추가
req = Request( WIKI_URL, 
               headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'})

In [4]:
## BS 변환 
html    = urlopen(req)
bs      = BeautifulSoup(html, 'html.parser')

## 모든 a 태그 추출
atag_list   = bs.find_all('a')

## 모든 a 태그의 href 추출
print('<a> 태그 개수: ', len(atag_list))
for link in atag_list:
    if 'href' in link.attrs:
        print(link['href'])

<a> 태그 개수:  194
#bodyContent
/wiki/%EC%9C%84%ED%82%A4%EB%AC%B8%ED%97%8C:%EB%8C%80%EB%AC%B8
/wiki/%EC%9C%84%ED%82%A4%EB%AC%B8%ED%97%8C:%EC%82%AC%EC%9A%A9%EC%9E%90_%EB%AA%A8%EC%9E%84
/wiki/%ED%8A%B9%EC%88%98:%EC%B5%9C%EA%B7%BC%EB%B0%94%EB%80%9C
/wiki/%ED%8A%B9%EC%88%98:%EB%AA%A8%EB%93%A0%EB%AC%B8%EC%84%9C
/wiki/%ED%8A%B9%EC%88%98:%EC%9E%84%EC%9D%98%EC%B5%9C%EC%83%81%EC%9C%84%EB%AC%B8%EC%84%9C/Main
/wiki/%ED%8A%B9%EC%88%98:%EC%9E%84%EC%9D%98%EB%AC%B8%EC%84%9C/Author
/wiki/%ED%8A%B9%EC%88%98:%EC%9E%84%EC%9D%98%EB%AC%B8%EC%84%9C/Portal
/wiki/%EC%9C%84%ED%82%A4%EB%AC%B8%ED%97%8C:%EB%8F%84%EC%9B%80%EB%A7%90
/wiki/%EC%9C%84%ED%82%A4%EB%AC%B8%ED%97%8C:%EB%8C%80%EB%AC%B8
/wiki/%ED%8A%B9%EC%88%98:%EA%B2%80%EC%83%89
https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=ko.wikisource.org&uselang=ko
/w/index.php?title=%ED%8A%B9%EC%88%98:%EA%B3%84%EC%A0%95%EB%A7%8C%EB%93%A4%EA%B8%B0&returnto=%EC%A0%80%EC%9E%90%3A%EC%9C%A4%EB%8F%99%EC%A3%BC
/w/index.php?title=%ED%8A%B9%EC%88%

In [5]:
from urllib.request import urlopen, Request
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd

# 1) 기본 URL
WIKI_URL = "https://ko.wikisource.org/wiki/%EC%A0%80%EC%9E%90:%EC%9C%A4%EB%8F%99%EC%A3%BC"

# 2) 공통 헤더
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def get_soup(url: str) -> BeautifulSoup:
    req = Request(url, headers=HEADERS)
    res = urlopen(req)
    return BeautifulSoup(res, "html.parser")

soup = get_soup(WIKI_URL)


In [6]:
#mw-content-text > div.mw-parser-output > ul
base = soup.select_one("#mw-content-text > div.mw-content-ltr.mw-parser-output")

poem_links = []

# 1) 모든 ul 안의 li > a 링크를 돈다
for ul in base.select("ul"):
    for a in ul.select("li > a"):
        title = a.get_text(strip=True)
        href = a.get("href", "")

        # 위키 내부 링크만 사용 (시가 아닌 메뉴 링크, 파일 링크 등은 대충 필터)
        if not href.startswith("/wiki/"):
            continue
        if ":" in href:   # '저자:', '파일:' 등은 제외
            continue

        full_url = urljoin(WIKI_URL, href)

        poem_links.append({
            "title": title,
            "url": full_url
        })

len(poem_links), poem_links[:5]


(96,
 [{'title': '하늘과 바람과 별과 시',
   'url': 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1948%EB%85%84)'},
  {'title': '하늘과 바람과 별과 시',
   'url': 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1955%EB%85%84)'},
  {'title': '하늘과 바람과 별과 시',
   'url': 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1979%EB%85%84)'},
  {'title': '서시',
   'url': 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1948%EB%85%84)/%EC%84%9C%EC%8B%9C'},
  {'title': '자화상',
   'url': 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1948%EB%85%84)/%EC%9E%90%ED%99%94%EC%83%81'}])

In [7]:
poems = []

for item in poem_links:
    title = item["title"]
    url = item["url"]

    detail = get_soup(url)
    body = detail.select_one("#mw-content-text > div.mw-parser-output")

    if body is None:
        content = ""
    else:
        # 줄바꿈 기준으로 본문 전체 텍스트 뽑기
        content = body.get_text("\n", strip=True)

    poems.append({
        "제목": title,
        "내용": content,
        "URL": url
    })

len(poems)


96